# Processamento de Linguagem Natural (PLN)
_Natural Language Processing (NLP) em Inglês_

## Análise de Sentimentos - Twitter

Identificar se o tweet é **racista/sexista** ou **não**

In [59]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk # pip install nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Ler dados

In [60]:
train = pd.read_csv('twitter/train.csv')
test = pd.read_csv('twitter/test.csv')

In [61]:
print(f'Train Shape: {train.shape}')
print(f'Test Shape: {test.shape}')

Train Shape: (31962, 3)
Test Shape: (17197, 2)


In [62]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [63]:
train['tweet'] = train['tweet'].str.lower() 
                        # deixar todas as strings em minusculo
                        # caso queira, dá pra deixar todas as strings em
                        # maiusculo usando train['tweet'].str.upper()

## Pré-processamento de Dados

### Tirar pontuações e caracteres especiais

In [64]:
# Expressão Regular
# r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"
train['tweet'] = train['tweet'].apply(lambda x: \
                     re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", \
                            "", x))

In [65]:
def limpa_texto(data, col):
    """
    Tira Pontuações e Caracteres especiais recebendo um dataframe (data) 
    e uma coluna (col).
    """
    data[col] = data[col].str.lower() 
    data[col] = data[col].apply(lambda x: \
                     re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", \
                            "", x))

In [66]:
limpa_texto(test, 'tweet')

In [67]:
test.head()

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,white supremacists want everyone to see the ...
2,31965,safe ways to heal your acne altwaystoheal h...
3,31966,is the hp and the cursed child book up for res...
4,31967,3rd bihday to my amazing hilarious nephew el...


### Remover palavras comuns

In [68]:
palavras_comuns = stopwords.words('english')

In [69]:
len(palavras_comuns)

179

In [70]:
# dividir os tweets em listas de palavras para que possamos
# retirar as stopwords, que são palavras que não acrescentam
# significado a nossa avaliação.
[palavra for palavra in train['tweet'].str.split()[0] if palavra not in palavras_comuns]

['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'run']

In [71]:
def remove_palavras_comuns(data, col, lang):
    """
    Remove stopwords do texto.
    """
    palavras_comuns = stopwords.words(lang)
    lista_de_lista_de_palavras = data[col].str.split()
    
    for idx, _ in data.iterrows():
        data[col].at[idx] = [palavra for palavra in \
                             lista_de_lista_de_palavras[idx] \
                             if palavra not in palavras_comuns]

In [72]:
remove_palavras_comuns(test, 'tweet', 'english')

In [73]:
remove_palavras_comuns(train, 'tweet', 'english')

### Remover variações da mesma palavra

In [74]:
tira_variacao = WordNetLemmatizer()

In [75]:
# exemplo de uso
tira_variacao.lemmatize('went','v')

'go'

In [76]:
def tira_variacao_de_palavras(data, col):
    tira_variacao = WordNetLemmatizer()
    for idx, _ in data.iterrows():
        data[col].at[idx] = [tira_variacao.lemmatize(palavra,'v') \
                             for palavra in data[col][idx]]

In [77]:
tira_variacao_de_palavras(train, 'tweet')

In [78]:
tira_variacao_de_palavras(test, 'tweet')

### Engenharia de Recursos (Feature Engineering ) 
#### Quantidade de palavras por tweet

In [79]:
train['palavras'] = train['tweet'].apply(len)

In [80]:
train['tweet']

0        [father, dysfunctional, selfish, drag, kid, dy...
1        [thank, lyft, credit, cant, use, cause, dont, ...
2                                        [bihday, majesty]
3                      [model, love, u, take, u, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
31957                                    [eat, isz, youuu]
31958    [see, nina, turner, airwaves, try, wrap, mantl...
31959    [listen, sad, songs, monday, morning, otw, wor...
31960    [sikh, temple, vandalise, calgary, wso, condem...
31961                                      [thank, follow]
Name: tweet, Length: 31962, dtype: object

In [81]:
test['palavras'] = test['tweet'].apply(len)

In [83]:
train.head()

,id,label,tweet,palavras
0,1,0,"[father, dysfunctional, selfish, drag, kid, dy...",7
1,2,0,"[thank, lyft, credit, cant, use, cause, dont, ...",13
2,3,0,"[bihday, majesty]",2
3,4,0,"[model, love, u, take, u, time, ur]",7
4,5,0,"[factsguide, society, motivation]",3


## Análise Exploratória de Dados (AED)

### Distribuição das quantidades de palavras por "label"

### Quantidade de cada "label"

### Palavras frequêntes

#### Averiguar palavras mais frequêntes classificadas como "racistas ou sexistas"

### Nuvem de Palavras 

### Retornar para formato de texto

###  Solucionar problema de desequilibrio dos dados

## Treinar Modelo

## Avaliar Performance do Modelo

## Submissão

Fontes (em inglês):
* http://bit.ly/NLP-Twitter
* http://bit.ly/NLP-Twitter-Competition